In [17]:
import geopandas as gpd
from shapely.geometry import Point

In [9]:
import pandas as pd
import re as re

In [32]:
conflict = pd.read_csv('Data\\conflict_data\\DRC_2.csv', header = 0, index_col = 0)
aid = pd.read_csv('Data\\aid_data\\data\\level_1a.csv')

# Extract ADM1 for aid data
def extract_adm_1(x):
    location = x.split('|')
    try:
        return location[3]
    except:
        return 'NA'
# add extra column with data on ADM1 locations
aid['adm_1'] = aid['gazetteer_adm_name'].apply(extract_adm_1)


In [347]:
# replace region with province for unifrom coding between aid and conflict datasets
conflict = conflict[pd.notnull(conflict['adm_1'])]
conflict['adm_1'] = conflict['adm_1'].apply(lambda x: x.replace('region', 'province'))

In [353]:
grouped_aid = aid.groupby('adm_1').count().iloc[:, 0]
grouped_conflict = conflict.groupby('adm_1').count().iloc[:, 0]

In [354]:
grouped_conflict

adm_1
Bandundu province       6
Bas-Congo              33
Eastern               601
Kasaï Occidental       26
Kasaï Oriental         31
Katanga               192
Kinshasa               68
Maniema                51
Nord Kivu            1152
Sud Kivu              549
Équateur               87
Name: year, dtype: int64

In [355]:
grouped_aid[grouped_aid > 10]

adm_1
Bandundu Province               148
Eastern                         135
Katanga                         208
Kinshasa                        189
NA                               99
Nord Kivu                       189
Province de l’Équateur          172
Province du Bas-Congo           103
Province du Kasaï-Occidental     95
Province du Kasaï-Oriental       98
Province du Maniema              74
Sud Kivu                        186
Name: project_id, dtype: int64

In [351]:
# TODO: 

def recode_provinces(x):
    pattern_1 = re.compile(r'(Tanganyika|Haut-Lomami|Lualaba|Haut-Katanga|Katanga|Shaba)') # recode to Katanga
    pattern_2 = re.compile(r'(Lomami|Sankuru|Kasaï-Oriental)') # recode to Kasaï-Oriental
    pattern_3 = re.compile(r'(Kasaï-Central)') # recode to Kasaï-Occidental
    pattern_4 = re.compile(r'(Kwango|Kwilu|Mai-Ndombe)') # recode to Bandundu
    pattern_5 = re.compile(r'(Bas-Uélé|Haut-Uélé|Ituri|Haut-Zaïre|Haut Congo|Orientale|Eastern)') # recode to Eastern
    pattern_6 = re.compile(r'(Bas Congo|Bas Zaïre)') # recode to Bas-Congo
    pattern_7 = re.compile(r'(Kasaï-Occidental|Kasaï Occidental)') # recode to Kasaï Occidental
    
    pattern_8 = re.compile(r'(Bandunu)') # recode to Bandunu
    pattern_9 = re.compile(r'(Katanga)') # recode to Katanga
    pattern_10 = re.compile(r'(Kinshasa )') # recode to Kinshasa
    pattern_11 = re.compile(r'(Équateur )') # recode to Équateur
    pattern_12 = re.compile(r'(Nord Kivu)') # recode to Nord Kivu
    pattern_13 = re.compile(r'(Sud Kivu|South Kivu)') # recode to Sud Kivu
    pattern_14 = re.compile(r'(Maniema )') # recode to Maniema
    pattern_15 = re.compile(r'(Kasaï Oriental|Kasaï-Oriental)') # recode to Kasaï Oriental

    if re.match(pattern_1, x):
        return 'Katanga'
    elif re.match(pattern_2, x):
        return 'Kasaï Oriental'
    elif re.match(pattern_3, x):
        return 'Kasaï Occidental'
    elif re.match(pattern_4, x):
        return 'Bandundu'
    elif re.match(pattern_5, x):
        return 'Eastern'
    elif re.match(pattern_6, x):
        return 'Bas-Congo'
    elif re.match(pattern_7, x):
        return 'Kasaï Occidental'
    
    elif re.match(pattern_8, x):
        return 'Bandunu'
    elif re.match(pattern_9, x):
        return 'Katanga'
    elif re.match(pattern_10, x):
        return 'Kinshasa'
    elif re.match(pattern_11, x):
        return 'Équateur'  
    elif re.match(pattern_12, x):
        return 'Nord Kivu'
    elif re.match(pattern_13, x):
        return 'Sud Kivu'
    elif re.match(pattern_14, x):
        return 'Maniema'
    elif re.match(pattern_15, x):
        return 'Kasaï Oriental'    
    return x

conflict['adm_1'] = conflict['adm_1'].apply(recode_provinces)
aid['adm_1'] = aid['adm_1'].apply(recode_provinces)

# Recode the provinces in such a way that they match each other:
# e.g. Easter province = Orientale province
# aid data has 11 provinces(old way ADMs)
# conflict data uses new provinces. Solution: recode conflict data to 11 old provinces (using wikipedia)

In [207]:
pattern = re.compile(r'(Bas-Uélé|Haut-Uélé|Ituri)')
for i in pattern.finditer('Kasaï Oriental province, Kasaï-Central province'):
    print(i)

In [367]:
# SOLVE THIS, doesn't match if looked at backwards
if re.match(r'Équateur', 'l asdasd Équateur'):
    print('Y')

In [370]:
# SOLVE THIS
if re.match(r'(Tanganyika|Haut-Lomami|Lualaba|Haut-Katanga)', 'Haut-Lomami province'):
    print('Y')

Y


In [262]:
conflict['adm_1'].head().apply(recode_provinces)

id
9607     Sud Kivu province
9735     Sud Kivu province
9868     Sud Kivu province
10068    Sud Kivu province
10424    Sud Kivu province
Name: adm_1, dtype: object

In [33]:
provinces = gpd.read_file('C:\\Users\\Yurii\\Desktop\\Konstanz stuff\\Diploma\\Data\\gadm36_COD_shp/gadm36_COD_1.shp')

In [35]:
geometry = [Point(xy) for xy in zip(conflict.longitude, conflict.latitude)]
gconflict = gpd.GeoDataFrame(conflict, crs = {'init': 'epsg:4326'}, geometry = geometry)

In [37]:
test = gpd.sjoin(gconflict, provinces, how="inner", op='intersects')

In [44]:
test[['NAME_1', 'deaths_civilians', 'where_coordinates', 'adm_1', 'geometry']].head()

,NAME_1,deaths_civilians,where_coordinates,adm_1,geometry
6803,Tshopo,4,DR Congo (Zaire),NaN,POINT (25 0)
20294,Tshopo,1,DR Congo (Zaire),NaN,POINT (25 0)
22978,Tshopo,40,Bafwasende location,Orientale province,POINT (27.266667 1.083333)
22983,Tshopo,36,Bafwasende zone,Haut-Zaïre region,POINT (27.266667 1.083333)
23011,Tshopo,0,Bengamisa locality,Haut-Zaïre region,POINT (25.211267 0.930883)
